# Personnalisation

**IMPORTANT** : Créez une ressource *Personalizer* et renseignez `SERVEUR` avec votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

**Paramétrage** dans *Gestion des ressources/Installation* :
- Durée d'attente de la récompense = 30s,
- Fréquence de mise à jour = 1min

In [1]:
from getpass import getpass

CLE = getpass()
SERVEUR = 'https://perso00.cognitiveservices.azure.com/'

In [2]:
from datetime import datetime
import random, requests
 
liste_menus = {
    'heure' : { 'titre'  : 'Heure' , 'action' : lambda: datetime.now().strftime('%x') },
    'date'  : { 'titre'  : 'Date'  , 'action' : lambda: datetime.now().strftime('%X') },
    'hasard': { 'titre'  : 'Hasard', 'action' : lambda: random.randint(0,100) },
    'monty' : { 'titre'  : 'Monty' , 'action' : lambda: "Ministry of silly walk" },
    'haiku' : { 'titre'  : 'Haiku' , 'action' : lambda: "Un vieil étang / Une grenouille qui plonge, / Le bruit de l'eau." },
    'star'  : { 'titre'  : 'Star'  , 'action' : lambda: ['Graham Chapman', 'John Cleese', 'Eric Idle', 'Michael Palin', 'Terry Jones', 'Terry Gilliam'][random.randint(0,5)] }
}

In [ ]:
from pprint import pprint
from IPython.display import clear_output

continuer = True
while continuer:
    # Demande de classement
    reponse = requests.post(
        url = SERVEUR + '/personalizer/v1.0/rank',
        headers = {
            'Ocp-Apim-Subscription-Key': CLE,
            'Content-Type': 'application/json'
        },
        json = {
            'contextFeatures': [ {'moment':'matin'}, {'lancement':'icone'}],
            'actions': [{ 'id': menu, 'features': [{'type': menu }] } for menu in liste_menus ]
        }
    )
    resultat = reponse.json()
    evenement = resultat['eventId']
    
    # Tri et affichage du menu proposé
    clear_output()
    classement = sorted(resultat['ranking'], key=lambda e:-e['probability'])
    i = 1
    for menu in classement:
        print("%d. %s (%.2f)"%(i, liste_menus[menu['id']]['titre'], menu['probability']), flush=True)
        i += 1
    print(f'{i}. Quitter', flush=True)
    choix = int(input("Votre choix ? "))
    
    # Envoi de la récompense/punition
    if choix == i:
        continuer = False
    else:
        print(liste_menus[classement[choix-1]['id']]['action']())
        recompense = 1 if choix==1 else 0.2-0.2*choix
        reponse = requests.post(
            url = f'{SERVEUR}/personalizer/v1.0/events/{evenement}/reward',
            headers = {
                'Ocp-Apim-Subscription-Key': CLE,
                'Content-Type': 'application/json'
            },
            json = { 'value': recompense }
        )
        print(f'(récompense {recompense} ({reponse})')

Tout effacer :

In [ ]:
for suppr in ['logs', 'model']:
    reponse = requests.delete(
        url = f'{SERVEUR}/personalizer/v1.0/{suppr}',
        headers = { 'Ocp-Apim-Subscription-Key': CLE }
    )
    print(f'{suppr} : {reponse}')
        